# KuiperHunter Model Inference Demo
This notebook demonstrates the trained 3D U-Net detecting injected movers.

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import yaml
import matplotlib.pyplot as plt
import numpy as np
from src.models.unet3d import UNet3D
from src.models.dataset import SimulationDataset

In [ ]:
# Load Utils
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

# Load Config
with open('../config/smoke_test.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
# Override for demo visibility
config['magnitude_range'] = [10.0, 10.0]
config['sequence_length'] = 8

# Load Model
model = UNet3D(n_channels=1, n_classes=1).to(device)
# Try to load checkpoint if exists, otherwise random init (just to check pipeline)
try:
    checkpoint_path = '../data/checkpoints/model_epoch_5.pth'
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print("Loaded checkpoint.")
except FileNotFoundError:
    print("Checkpoint not found, using random weights.")
    
model.eval();

In [ ]:
# Generate Sample
dataset = SimulationDataset(config, epoch_size=1)
img_tensor, mask_tensor = dataset[0]

# Run Inference
with torch.no_grad():
    input_batch = img_tensor.unsqueeze(0).to(device)
    logits = model(input_batch)
    preds = torch.sigmoid(logits)
    
input_np = input_batch.cpu().numpy()[0, 0]
pred_np = preds.cpu().numpy()[0, 0]
mask_np = mask_tensor.numpy()[0]

In [ ]:
# Visualize Frame 4
idx = 4
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title("Input Frame 4")
plt.imshow(input_np[idx], cmap='gray')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.title("Ground Truth")
plt.imshow(mask_np[idx], cmap='gray')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.title("Prediction Probability")
plt.imshow(pred_np[idx], cmap='inferno')
plt.colorbar()
plt.show()